In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK：用于导出到边缘的AutoML训练图像对象检测模型

<table align="left">
  <td> 
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_image_object_detection_online_export_edge.ipynb"> 
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行 
    </a> 
  </td> 
  <td> 
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_image_object_detection_online_export_edge.ipynb"> 
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"> 查看GitHub
    </a> 
  </td> 
  <td> 
    <a href="https://console.cloud.google.com/ai/platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_image_object_detection_online_export_edge.ipynb"> 
      在Google Cloud Notebooks中打开 
    </a> 
  </td> 
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Vertex SDK创建图像目标检测模型，然后将其导出为边缘模型，使用Google Cloud AutoML模型。

### 数据集

本教程使用的数据集是来自[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/overview)中的[OpenImages数据集](https://www.tensorflow.org/datasets/catalog/open_images_v4)中的沙拉类别。这个数据集不需要进行任何特征工程。在本教程中使用的数据集版本存储在一个公共的Cloud Storage存储桶中。训练好的模型可以预测图像中沙拉类别的边界框位置和相应的类型，包括五种物品：沙拉、海鲜、番茄、烘焙食品或奶酪。

### 目标

在本教程中，您将使用Vertex SDK从Python脚本创建一个AutoML图像对象检测模型，然后将该模型导出为TFLite格式的Edge模型。您也可以使用`gcloud`命令行工具或在云控制台中在线创建AutoML模型。

执行的步骤包括：

- 创建一个Vertex `Dataset`资源。
- 训练模型。
- 从`Model`资源中将`Edge`模型导出到Cloud Storage。
- 在本地下载模型。
- 进行本地预测。

### 成本

本教程使用谷歌云的付费组件:

* Vertex AI
* 云存储

了解[Vertex AI
价格](https://cloud.google.com/vertex-ai/pricing) 和 [云存储
价格](https://cloud.google.com/storage/pricing)，并使用 [定价
计算器](https://cloud.google.com/products/calculator/)
根据您的预期使用量生成成本估算。

### 设置您的本地开发环境

如果您正在使用Colab或Google Cloud笔记本，您的环境已经满足运行此笔记本的所有要求。您可以跳过此步骤。

否则，请确保您的环境符合此笔记本的要求。您需要以下内容：

- 云存储SDK
- Git
- Python 3
- virtualenv
- 在使用Python 3的虚拟环境中运行的Jupyter笔记本

[设置Python开发环境](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了一套简明的指令：

1. [安装并初始化SDK](https://cloud.google.com/sdk/docs/)。

2. [安装Python 3](https://cloud.google.com/python/setup#installing_python)。

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，在终端窗口中运行`pip3 install jupyter`。

5. 要启动Jupyter，在终端窗口中运行`jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开此笔记本。

## 安装

安装最新版本的Python Vertex SDK。

In [ ]:
import os

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

安装最新的*google-cloud-storage*库的GA版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

In [ ]:
if os.environ["IS_TESTING"]:
    ! pip3 install --upgrade tensorflow $USER_FLAG

重新启动内核

安装了额外的包之后，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU运行时

本教程不需要GPU运行时。

### 设置您的Google Cloud项目

**无论您使用的是哪种笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。您首次创建帐户时，将获得300美元的免费信用额用于计算/存储成本。

2. [确保您的项目已启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下API：Vertex AI API、Compute Engine API和Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您在本地运行此笔记本，您需要安装[Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格以确保Cloud SDK对本笔记本中的所有命令使用正确的项目。

**注意**: Jupyter会执行以`!`为前缀的行作为shell命令，并且会插值以`$`为前缀的Python变量。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 地区

您还可以更改“REGION”变量，该变量用于本笔记本其余部分的操作。以下是Vertex AI支持的地区。我们建议您选择距离您最近的地区。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您不能使用多区域存储桶进行与Vertex AI的训练。并非所有地区均为所有Vertex AI服务提供支持。

了解有关[Vertex AI地区](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在参加直播教程会话，可能会使用共享测试账户或项目。为了避免用户在创建的资源之间发生名称冲突，您可以为每个实例会话创建一个时间戳，并将时间戳附加到您在本教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的Google Cloud帐户

**如果您正在使用Google Cloud笔记本**，您的环境已经经过验证。跳过这一步。

**如果您正在使用Colab**，运行下面的单元格，并按照提示进行身份验证以通过oAuth认证您的帐户。

**否则**，请按照以下步骤操作：

在Cloud控制台中，转到[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐户**。

在**服务帐户名称**字段中，输入一个名称，然后点击**创建**。

在**授予此服务帐户对项目的访问权限**部分，点击角色下拉列表。在过滤框中输入“Vertex”，并选择**Vertex管理员**。在过滤框中输入“存储对象管理员”，并选择**存储对象管理员**。

点击创建。包含您的密钥的JSON文件将下载到您的本地环境中。

在下面的单元格中，输入您服务帐户密钥的路径作为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

创建一个云存储桶

**以下步骤是必需的，无论您使用的是哪种笔记本环境。**

当您初始化用于 Python 的 Vertex SDK 时，您需要指定一个云存储暂存桶。暂存桶是在各个会话中保留与数据集和模型资源相关的所有数据的位置。

在下方设置您的云存储桶的名称。存储桶的名称必须在所有 Google Cloud 项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只要您的存储桶尚不存在：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查其内容来验证对您的云存储桶的访问。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip

使用Python为您的项目和相应的存储桶初始化Vertex SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

# 教程

现在您已经准备好开始创建自己的AutoML图像对象检测模型了。

云存储培训数据的位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-samples-data/vision/salads.csv"

快速查看您的数据

本教程使用存储在公共云存储桶中的沙拉数据集的一个版本，使用一个CSV索引文件。

从快速查看数据开始。通过计算CSV索引文件中的行数（`wc -l`）来计算示例的数量，然后查看前几行数据。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 创建数据集

接下来，使用`ImageDataset`类的`create`方法创建`Dataset`资源，该方法接受以下参数：

-`display_name`：`Dataset`资源的可读名称。
-`gcs_source`：要将数据项导入`Dataset`资源的一个或多个数据集索引文件列表。
-`import_schema_uri`：数据项的数据标记模式。

此操作可能需要几分钟时间。

In [ ]:
dataset = aip.ImageDataset.create(
    display_name="Salads" + "_" + TIMESTAMP,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aip.schema.dataset.ioformat.image.bounding_box,
)

print(dataset.resource_name)

### 创建和运行训练管道

要训练一个AutoML模型，您需要执行两个步骤：1）创建一个训练管道，2）运行这个管道。

#### 创建训练管道

使用`AutoMLImageTrainingJob`类创建一个AutoML训练管道，具有以下参数：

- `display_name`：`TrainingJob`资源的可读名称。
- `prediction_type`：为模型训练指定的任务类型。
  - `classification`：一个图像分类模型。
  - `object_detection`：一个图像目标检测模型。
- `multi_label`：如果是分类任务，指定单标签（`False`）还是多标签（`True`）。
- `model_type`：部署模型的类型。
  - `CLOUD`：在Google Cloud上部署。
  - `CLOUD_HIGH_ACCURACY_1`：优化准确性而非延迟，在Google Cloud上部署。
  - `CLOUD_LOW_LATENCY_`：优化延迟而非准确性，在Google Cloud上部署。
  - `MOBILE_TF_VERSATILE_1`：在边缘设备上部署。
  - `MOBILE_TF_HIGH_ACCURACY_1`：优化准确性而非延迟，在边缘设备上部署。
  - `MOBILE_TF_LOW_LATENCY_1`：优化延迟而非准确性，在边缘设备上部署。
- `base_model`：（可选）从现有的`Model`资源进行迁移学习，仅支持图像分类。

实例化的对象是训练作业的DAG（有向无环图）。

In [ ]:
dag = aip.AutoMLImageTrainingJob(
    display_name="salads_" + TIMESTAMP,
    prediction_type="object_detection",
    multi_label=False,
    model_type="MOBILE_TF_LOW_LATENCY_1",
    base_model=None,
)

print(dag)

#### 运行训练管道

接下来，你可以运行DAG以启动训练作业，通过调用`run`方法，并传入以下参数：

- `dataset`: 用于训练模型的`Dataset`资源。
- `model_display_name`: 训练模型的可读名称。
- `training_fraction_split`: 用于训练的数据集百分比。
- `test_fraction_split`: 用于测试（留出数据）的数据集百分比。
- `validation_fraction_split`: 用于验证的数据集百分比。
- `budget_milli_node_hours`: （可选）指定的最大训练时间，单位为毫小时（1000 = 小时）。
- `disable_early_stopping`: 如果为`True`，服务可能认为在使用整个预算之前训练已经完成，不能进一步改进模型的目标测量。

完成后，`run`方法会返回`Model`资源。

训练管道的执行可能需要最多20分钟。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="salads_" + TIMESTAMP,
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    budget_milli_node_hours=20000,
    disable_early_stopping=False,
)

## 查看模型评估分数
在你的模型训练完成后，你可以查看它的评估分数。

首先，你需要获取新模型的引用。与数据集一样，你可以使用部署模型时创建的模型变量的引用，或者列出项目中的所有模型。

In [ ]:
# Get model resource ID
models = aip.Model.list(filter="display_name=salads_" + TIMESTAMP)

# Get a reference to the Model Service client
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
model_service_client = aip.gapic.ModelServiceClient(client_options=client_options)

model_evaluations = model_service_client.list_model_evaluations(
    parent=models[0].resource_name
)
model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

## 导出为Edge模型

您可以将AutoML图像对象检测模型导出为`Edge`模型，然后可以将其自定义部署到边缘设备或下载到本地。使用`export_model()`方法将模型导出到云存储，需要以下参数：

- `artifact_destination`：要将SavedFormat模型工件存储到的云存储位置。
- `export_format_id`：要将模型格式保存为的格式。对于AutoML图像对象检测，只有一个选项：
   - `tf-saved-model`：部署到容器使用的TensorFlow SavedFormat。
   - `tflite`：部署到边缘设备或移动设备的TensorFlow Lite。
   - `edgetpu-tflite`：用于TPU的TensorFlow Lite。
   - `tf-js`：Web客户端使用的TensorFlow。
   - `coral-ml`：用于Coral设备。

- `sync`：是否同步或异步执行操作。

In [ ]:
response = model.export_model(
    artifact_destination=BUCKET_NAME, export_format_id="tflite", sync=True
)

model_package = response["artifactOutputUri"]

#### 下载 TFLite 模型文件

现在您已经导出了您的模型的 TFLite 版本，您可以在本地测试导出的模型，但首先需要从云存储中下载它。

In [ ]:
! gsutil ls $model_package
# Download the model artifacts
! gsutil cp -r $model_package tflite

tflite_path = "tflite/model.tflite"

实例化一个TFLite解释器

模型的TFLite版本不是TensorFlow SavedModel格式。您不能直接使用像predict()这样的方法。相反，要使用TFLite解释器。您必须首先按照以下步骤为TFLite模型设置解释器：

- 为TFLite模型实例化一个TFLite解释器。
- 指示解释器为模型分配输入和输出张量。
- 获取有关需要了解以进行预测的模型输入和输出张量的详细信息。

In [ ]:
import tensorflow as tf

interpreter = tf.lite.Interpreter(model_path=tflite_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]["shape"]

print("input tensor shape", input_shape)

### 获取测试项目

您将从数据集中随机选择一个示例作为测试项目。不必担心该示例很可能已经在训练模型时使用过 -- 我们只是想演示如何进行预测。

In [ ]:
test_items = ! gsutil cat $IMPORT_FILE | head -n1
test_item = test_items[0].split(",")[0]

with tf.io.gfile.GFile(test_item, "rb") as f:
    content = f.read()
test_image = tf.io.decode_jpeg(content)
print("test image shape", test_image.shape)

test_image = tf.image.resize(test_image, (224, 224))
print("test image shape", test_image.shape, test_image.dtype)

test_image = tf.cast(test_image, dtype=tf.uint8).numpy()

使用TFLite模型进行预测

最后，使用您的TFLite模型进行预测，操作如下：

- 将测试图片转换为包含单个图片的批次（`np.expand_dims`）。
- 将输入张量设置为解释器的单个图片批次（`data`）。
- 调用解释器。
- 检索预测的softmax概率（`get_tensor`）。
- 确定哪个标签具有最高的概率（`np.argmax`）。

In [ ]:
import numpy as np

data = np.expand_dims(test_image, axis=0)

interpreter.set_tensor(input_details[0]["index"], data)

interpreter.invoke()

softmax = interpreter.get_tensor(output_details[0]["index"])

label = np.argmax(softmax)

print(label)

清理

要清理此项目中使用的所有谷歌云资源，您可以删除用于教程的[谷歌云项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 终端
- AutoML训练作业
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储存储桶

In [ ]:
delete_all = True

if delete_all:
    # Delete the dataset using the Vertex dataset object
    try:
        if "dataset" in globals():
            dataset.delete()
    except Exception as e:
        print(e)

    # Delete the model using the Vertex model object
    try:
        if "model" in globals():
            model.delete()
    except Exception as e:
        print(e)

    # Delete the endpoint using the Vertex endpoint object
    try:
        if "endpoint" in globals():
            endpoint.delete()
    except Exception as e:
        print(e)

    # Delete the AutoML or Pipeline trainig job
    try:
        if "dag" in globals():
            dag.delete()
    except Exception as e:
        print(e)

    # Delete the custom trainig job
    try:
        if "job" in globals():
            job.delete()
    except Exception as e:
        print(e)

    # Delete the batch prediction job using the Vertex batch prediction object
    try:
        if "batch_predict_job" in globals():
            batch_predict_job.delete()
    except Exception as e:
        print(e)

    # Delete the hyperparameter tuning job using the Vertex hyperparameter tuning object
    try:
        if "hpt_job" in globals():
            hpt_job.delete()
    except Exception as e:
        print(e)

    if "BUCKET_NAME" in globals():
        ! gsutil rm -r $BUCKET_NAME